# Exploit or Points of Interest
Builds and populates the `exploit` table from a locally stored CSV.

### Import libraries

In [1]:
# Basic stack
import os
import shutil
from glob import glob
import warnings

# Web stack
import json

# Database stack
import sqlite3

# Data Science stack
import numpy as np
import pyproj
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import utm
import pandas as pd
import geopandas as gpd
import shapely
from shapely import to_geojson
from shapely.ops import transform
from shapely.geometry import Polygon
from fiona.drvsupport import supported_drivers
import folium

### Set Environmental Variables

In [2]:
warnings.filterwarnings("ignore")

### User defined variables

In [3]:
db = "../../db.sqlite3"
labels_csv = "../../data/databases/labels.csv"

### User defined functions

In [4]:
def insert_exploit(c, cat_id, dar_id, poi, sample_idx, lat, lon, email,
                              client_ip, out_time, in_time, user_label, confidence,
                              species, comments):
    """ Inserts Points of Interest into the Exploit table.

        C - A cursor
        DAR ID - An Entity ID value from EarthExplorer
    """
    sql_string = f"INSERT INTO exploit(id, dar, poi, sample_idx, latitude, longitude, email, " +\
                    f"client_ip, out_time, in_time, user_label, confidence, species, comments) VALUES (" +\
                    f"\"{cat_id}\", \"{dar_id}\", \"{poi}\", \"{sample_idx}\", \"{lat}\", \"{lon}\"," +\
                    f"\"{email}\", \"{client_ip}\", \"{out_time}\", \"{in_time}\", \"{user_label}\"," +\
                    f"\"{confidence}\", \"{species}\", \"{comments}\")"
    c.execute(sql_string)

### Load data, decat, and reorganize data

In [5]:
# Load
df = pd.read_csv(labels_csv)
print("Your initial DAR spreadsheet is {} in size".format(df.shape))

# Deconcactenate 
df['dar'] = df['sample_idx'].apply(lambda x: x.split('-')[1])
df['catid'] = df['sample_idx'].apply(lambda x: x.split('-')[3])
df['poi'] = df['sample_idx'].apply(lambda x: x.split('-')[4])

# Rearange columns
df = df.iloc[:,[12, 11, 13, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
df.head(5)

Your initial DAR spreadsheet is (921, 11) in size


,catid,dar,poi,sample_idx,latitude,longitude,email,client_ip,out_time,in_time,user_label,confidence,species,comments
0,104001005A503D00,0680,382,DAR-0680-20MAY28-104001005A503D00-382,40.641478,-70.428225,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:22 2023,Fri May 12 22:11:26 2023,unsure,Possible,Unknown,NaN
1,104001005A503D00,0680,392,DAR-0680-20MAY28-104001005A503D00-392,40.641705,-70.404564,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:26 2023,Fri May 12 22:11:28 2023,unsure,Possible,Unknown,NaN
2,104001005A503D00,0680,684,DAR-0680-20MAY28-104001005A503D00-684,40.640648,-70.384945,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:28 2023,Fri May 12 22:13:50 2023,unsure,Possible,Unknown,NaN
3,104001005A503D00,0680,2187,DAR-0680-20MAY28-104001005A503D00-2187,40.636506,-70.413323,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:13:50 2023,Fri May 12 22:13:51 2023,unsure,Possible,Unknown,NaN
4,104001005A503D00,0680,2418,DAR-0680-20MAY28-104001005A503D00-2418,40.636893,-70.341557,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:13:51 2023,Fri May 12 22:13:52 2023,unsure,Possible,Unknown,NaN


### Drop table
This is for demonstration and troubleshooting purposes.

In [6]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS exploit''')
conn.commit()
conn.close()

### Create `exploit` table

In [7]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS exploit(
        id,
        vendor_id,
        entity_id,
        cid  VARCHAR(20),
        dar INTEGER,
        poi INTEGER,
        sample_idx VARCHAR(40),
        latitude NUMERIC(2, 6),
        longitude NUMERIC(3, 6),
        email VARCHAR(35),
        client_ip VARCHAR(13),
        out_time DATE,
        in_time DATE,
        user_label VARCHAR(10),
        confidence VARCHAR(10),
        species VARCHAR(25),
        comments VARCHAR(500),
        FOREIGN KEY (id)
            REFERENCES etl(id),
        FOREIGN KEY (vendor_id)
            REFERENCES etl(vendor_id),
        FOREIGN KEY (entity_id)
            REFERENCES etl(entity_id),
        FOREIGN KEY (cid)
            REFERENCES tasking(id),
        FOREIGN KEY (dar)
            REFERENCES tasking(dar)
    )
''')

conn.commit()
conn.close()

### Insert into `exploit` table

In [8]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in df.iterrows():
    try:
        cat_id = row['catid']
        dar_id = int(row['dar'])
        poi = int(row['poi'])
        sample_idx = row['sample_idx']
        lat = row['latitude']
        lon = row['longitude']
        email = row['email']
        client_ip = row['client_ip']
        out_time = row['out_time']
        in_time = row['in_time']
        user_label = row['user_label']
        confidence = row['confidence']
        species = row['species']
        comments = row['comments']
        try:
            insert_exploit(c, cat_id, dar_id, poi, sample_idx, lat, lon, email,
                              client_ip, out_time, in_time, user_label, confidence,
                              species, comments)
        except Exception as e:
            print("Exception: {} was raised for POI ID {}".format(e, poi))
    except Exception as e:
        print("Exception: {} was raised for iteration {}".format(e, i))
print("Done updating EXPLOIT table!")

conn.commit()
conn.close()

Done updating EXPLOIT table!


### Select new information, review it

In [9]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

df = pd.read_sql_query(f"SELECT * FROM exploit", conn)

conn.commit()
conn.close()

df.head()

,id,vendor_id,entity_id,cid,dar,poi,sample_idx,latitude,longitude,email,client_ip,out_time,in_time,user_label,confidence,species,comments
0,104001005A503D00,None,None,None,680,382,DAR-0680-20MAY28-104001005A503D00-382,40.641478,-70.428225,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:22 2023,Fri May 12 22:11:26 2023,unsure,Possible,Unknown,nan
1,104001005A503D00,None,None,None,680,392,DAR-0680-20MAY28-104001005A503D00-392,40.641705,-70.404564,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:26 2023,Fri May 12 22:11:28 2023,unsure,Possible,Unknown,nan
2,104001005A503D00,None,None,None,680,684,DAR-0680-20MAY28-104001005A503D00-684,40.640648,-70.384945,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:11:28 2023,Fri May 12 22:13:50 2023,unsure,Possible,Unknown,nan
3,104001005A503D00,None,None,None,680,2187,DAR-0680-20MAY28-104001005A503D00-2187,40.636506,-70.413323,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:13:50 2023,Fri May 12 22:13:51 2023,unsure,Possible,Unknown,nan
4,104001005A503D00,None,None,None,680,2418,DAR-0680-20MAY28-104001005A503D00-2418,40.636893,-70.341557,calebrob6@gmail.com,67.168.92.254,Fri May 12 22:13:51 2023,Fri May 12 22:13:52 2023,unsure,Possible,Unknown,nan


# End